In [1]:
# !pip install --force-reinstall "git+https://github.com/AEmanuelli/shimmer-ssd.git"

In [2]:

from shimmer.modules.domain import DomainModule
from shimmer.modules.global_workspace import GlobalWorkspace2Domains, SchedulerArgs

from shimmer_ssd.config import DomainModuleVariant, LoadedDomainConfig, load_config

from shimmer_ssd.modules.domains import load_pretrained_domains

from simple_shapes_dataset import SimpleShapesDataModule, get_default_domains


# %matplotlib inlineKikabosco2

import io
import math
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import torch
from ipywidgets import interact, interact_manual
from PIL import Image
from pathlib import Path
from shimmer_ssd.logging import attribute_image_grid
from shimmer_ssd.config import load_config
from torch.nn.functional import one_hot

from simple_shapes_dataset.cli import generate_image
%matplotlib widget

# And now we load the GW checkpoint
checkpoint_path = Path("/mnt/HD2/alexis_data/checkpoints_backup/checkpoints")

# We don't use cli in the notebook, but consider using it in normal scripts.
config = load_config("./config", use_cli=False)

config.domain_proportions = {
    frozenset(["v"]): 1.0,
    frozenset(["attr"]): 1.0,
    frozenset(["v", "attr"]): 1.0,
}

config.domains = [
    LoadedDomainConfig(
        domain_type=DomainModuleVariant.v_latents,
        checkpoint_path=checkpoint_path / "domain_v.ckpt",
    ),
    LoadedDomainConfig(
        domain_type=DomainModuleVariant.attr_legacy_no_color,
        checkpoint_path=checkpoint_path / "domain_attr.ckpt",
        args = {"alpha": 1, "temperature": 1},
    ),
]

config.domain_data_args["v_latents"]["presaved_path"] = "domain_v.npy"
config.global_workspace.latent_dim = 12





In [3]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import math
import io
from PIL import Image
from torch.nn.functional import one_hot
from ipywidgets import interact
import torch.nn.functional as F

# checkpoint = "/home/alexis/Desktop/checkpoints/training_logs/Removing_colors/High alpha 5 (color) - corrected v5 (Logsoftmax --> exp --> log)/checkpoints/last.ckpt"
# checkpoint = "/home/alexis/Desktop/checkpoints/training_logs/Removing_colors/Base_params - corrected v5 (Logsoftmax --> exp --> log)/checkpoints/last.ckpt"

# checkpoint = "/home/alexis/Desktop/checkpoints/training_logs/Removing_colors/(Seed 0) High_cycles (10) v5/checkpoints/last.ckpt"
checkpoint = "/home/alexis/Desktop/checkpoints/training_logs/Removing_colors/(Seed 0) High_cycles (10) v5/checkpoints/last.ckpt"

In [4]:



# config.global_workspace.encoders.hidden_dim = 256
# config.global_workspace.encoders.n_layers = 2
# config.global_workspace.decoders.hidden_dim = 256
# config.global_workspace.decoders.n_layers = 2
# # we load the pretrained domain modules and define the associated GW encoders and decoders
# domain_modules, gw_encoders, gw_decoders = load_pretrained_domains(
#     config.domains,
#     config.global_workspace.latent_dim,
#     config.global_workspace.encoders.hidden_dim,
#     config.global_workspace.encoders.n_layers,
#     config.global_workspace.decoders.hidden_dim,
#     config.global_workspace.decoders.n_layers,
# )
# global_workspace = GlobalWorkspace2Domains.load_from_checkpoint(
#     checkpoint,
#     domain_mods=domain_modules,
#     gw_encoders=gw_encoders,
#     gw_decoders=gw_decoders,
# )
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# global_workspace.to(device)

# cat2idx = {"Diamond": 0, "Egg": 1, "Triangle": 2}
# categories = ["Diamond", "Egg", "Triangle"]

# def normalize_cat_values(diamond, egg, triangle):
#     """Normalise les valeurs de catégories pour qu'elles somment à 1"""
#     values = torch.tensor([diamond, egg, triangle], dtype=torch.float32)
#     # Appliquer softmax pour garantir que les valeurs sont positives et somment à 1
#     return values

# def get_dominant_category(cat_values):
#     """Retourne la catégorie dominante pour la visualisation"""
#     cat_idx = torch.argmax(cat_values).item()
#     return categories[cat_idx]

# def get_image(cat_values, x, y, size, rot, color_r, color_g, color_b):
#     """Génère une image basée sur la catégorie dominante"""
#     # Obtenir la catégorie avec la valeur la plus élevée
#     cat_idx = torch.argmax(cat_values).item()
#     cat = categories[cat_idx]
    
#     # Générer l'image pour cette catégorie
#     fig, ax = plt.subplots(figsize=(32, 32), dpi=1)
#     generate_image(
#         ax,
#         cat2idx[cat],
#         [int(x * 18 + 7), int(y * 18 + 7)],
#         size * 7 + 7,
#         rot * 2 * math.pi,
#         np.array([color_r * 255, color_g * 255, color_b * 255]),
#         imsize=32,
#     )
#     ax.set_facecolor("black")
#     plt.tight_layout(pad=0)
    
#     # Retourner l'image
#     buf = io.BytesIO()
#     fig.savefig(buf)
#     buf.seek(0)
#     image = Image.open(buf)
#     plt.close(fig)
#     return image

# @interact(
#     diamond=(0.0001, 1, 0.00001),
#     egg=(0.0001, 1, 0.00001),
#     triangle=(0, 1, 0.00001),
#     x=(0, 1, 0.01),
#     y=(0, 1, 0.01),
#     rot=(0, 1, 0.01),
#     size=(0, 1, 0.01),
#     color_r=(0, 1, 0.01),
#     color_g=(0, 1, 0.01),
#     color_b=(0, 1, 0.01),
# )
# def play_with_gw(
#     diamond: float = 0.0,
#     egg: float = 0.0,
#     triangle: float = 1.0,
#     x: float = 0.5,
#     y: float = 0.5,
#     rot: float = 0.5,
#     size: float = 0.5,
#     color_r: float = 1,
#     color_g: float = 0,
#     color_b: float = 0,
# ):
#     # Normaliser les valeurs de catégories
#     cat_values = normalize_cat_values(diamond, egg, triangle)
#     dominant_cat = get_dominant_category(cat_values)
    
#     fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
#     # Image originale basée sur la catégorie dominante
#     image = get_image(cat_values, x, y, size, rot, color_r, color_g, color_b)
#     axes[0].set_facecolor("black")
#     axes[0].set_title(f"Image originale\nCatégorie dominante: {dominant_cat}")
#     axes[0].set_xticks([])
#     axes[0].set_yticks([])
#     axes[0].imshow(image)
    
#     # Visualisation des valeurs de catégories
#     axes[1].bar(categories, cat_values.numpy())
#     axes[1].set_title("Distribution des catégories")
#     axes[1].set_ylim(0, 1)
    
#     # Préparer les attributs pour le global workspace
#     rotx = math.cos(rot * 2 * math.pi)
#     roty = math.sin(rot * 2 * math.pi)
#     attributes = torch.tensor(
#         [[x * 2 - 1, y * 2 - 1, size * 2 - 1, rotx, roty]]
#     )
    
#     # Exécuter le modèle
#     # Utiliser directement le tenseur cat_values au lieu de one_hot encoding
#     category = cat_values.unsqueeze(0)  # Ajouter dimension batch
    
#     samples = [category.to(device), attributes.to(device)]
#     attr_gw_latent = global_workspace.gw_mod.encode({"attr": global_workspace.encode_domain(samples, "attr")})
#     gw_latent = global_workspace.gw_mod.fuse(
#         attr_gw_latent, {"attr": torch.ones(attr_gw_latent["attr"].size(0)).to(device)}
#     )
#     decoded_latents = global_workspace.gw_mod.decode(gw_latent)["v_latents"]
#     decoded_images = (
#         global_workspace.domain_mods["v_latents"]
#         .decode_images(decoded_latents)[0]
#         .permute(1, 2, 0)
#         .detach()
#         .cpu()
#         .numpy()
#     )
    
#     # Afficher l'image générée
#     axes[2].imshow(decoded_images)
#     axes[2].set_xticks([])
#     axes[2].set_yticks([])
#     axes[2].set_title("Image traduite via GW")
    
#     # Afficher les valeurs exactes
#     print(f"Valeurs normalisées: Diamond={cat_values[0]:.4f}, Egg={cat_values[1]:.4f}, Triangle={cat_values[2]:.4f}")
    
#     plt.tight_layout()
#     plt.show()

In [5]:
config.global_workspace.encoders.hidden_dim = 256
config.global_workspace.encoders.n_layers = 2
config.global_workspace.decoders.hidden_dim = 256
config.global_workspace.decoders.n_layers = 2
# we load the pretrained domain modules and define the associated GW encoders and decoders
domain_modules, gw_encoders, gw_decoders = load_pretrained_domains(
    config.domains,
    config.global_workspace.latent_dim,
    config.global_workspace.encoders.hidden_dim,
    config.global_workspace.encoders.n_layers,
    config.global_workspace.decoders.hidden_dim,
    config.global_workspace.decoders.n_layers,
)
global_workspace = GlobalWorkspace2Domains.load_from_checkpoint(
    checkpoint,
    domain_mods=domain_modules,
    gw_encoders=gw_encoders,
    gw_decoders=gw_decoders,
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
global_workspace.to(device)


cat2idx = {"Diamond": 0, "Egg": 1, "Triangle": 2} 

def get_image(cat, x, y, size, rot, color_r, color_g, color_b):
    fig, ax = plt.subplots(figsize=(32, 32), dpi=1)
    # The dataset generatoion tool has function to generate a matplotlib shape
    # from the attributes.
    generate_image(
        ax,
        cat2idx[cat],
        [int(x * 18 + 7), int(y * 18 + 7)],
        size * 7 + 7,
        rot * 2 * math.pi,
        np.array([color_r * 255, color_g * 255, color_b * 255]),
        imsize=32,
    )
    ax.set_facecolor("black")
    plt.tight_layout(pad=0)
    # Return this as a PIL Image.
    # This is to have the same dpi as saved images
    # otherwise matplotlib will render this in very high quality
    buf = io.BytesIO()
    fig.savefig(buf)
    buf.seek(0)
    image = Image.open(buf)
    plt.close(fig)
    return image


@interact(
    cat=["Triangle", "Egg", "Diamond"],
    x=(0, 1, 0.1),
    y=(0, 1, 0.1),
    rot=(0, 1, 0.1),
    size=(0, 1, 0.1),
    color_r=(0, 1, 0.1),
    color_g=(0, 1, 0.1),
    color_b=(0, 1, 0.1),
)
def play_with_gw(
    cat: str = "Triangle",
    x: float = 0.5,
    y: float = 0.5,
    rot: float = 0.5,
    size: float = 0.5,
    color_r: float = 1,
    color_g: float = 0,
    color_b: float = 0,
):
    fig, axes = plt.subplots(1, 2)
    image = get_image(cat, x, y, size, rot, color_r, color_g, color_b)
    axes[0].set_facecolor("black")
    axes[0].set_title("Original image from attributes")
    axes[0].set_xticks([])
    axes[0].set_yticks([])
    axes[0].imshow(image)

    # normalize the attribute for the global workspace.
    cat_values = one_hot(torch.tensor([cat2idx[cat]]), 3)
    # category = category.float().unsqueeze(0)  # Add batch dimension
    category = cat_values.float()# Add batch dimension
    # Add batch dimension and make one-hot a little noisy
    category = category.float()
    # Add a small noise to make the one-hot less strict
    noise = torch.rand_like(category) * 0.01
    category = category + noise
    # Normalize to ensure it still sums to 1
    category = category / category.sum()
    
    
    print(f"Category values: {category}")

    rotx = math.cos(rot * 2 * math.pi)
    roty = math.sin(rot * 2 * math.pi)
    attributes = torch.tensor(
        [[x * 2 - 1, y * 2 - 1, size * 2 - 1, rotx, roty]]# color_r * 2 - 1, color_g * 2 - 1, color_b * 2 - 1]])
    )
    samples = [category.to(device), attributes.to(device)]
    attr_gw_latent = global_workspace.gw_mod.encode({"attr": global_workspace.encode_domain(samples, "attr")})
    gw_latent = global_workspace.gw_mod.fuse(
        attr_gw_latent, {"attr": torch.ones(attr_gw_latent["attr"].size(0)).to(device)}
    )
    decoded_latents = global_workspace.gw_mod.decode(gw_latent)["v_latents"]
    decoded_images = (
        global_workspace.domain_mods["v_latents"]
        .decode_images(decoded_latents)[0]
        .permute(1, 2, 0)
        .detach()
        .cpu()
        .numpy()
    )
    axes[1].imshow(decoded_images)
    axes[1].set_xticks([])
    axes[1].set_yticks([])
    axes[1].set_title("Translated image through GW")
    plt.show()

default alpha and temperature are loaded (1, 1)


interactive(children=(Dropdown(description='cat', options=('Triangle', 'Egg', 'Diamond'), value='Triangle'), F…